In [1]:
import torch 
import torch.nn as nn  
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast
import cv2
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast
from torch.cuda import amp
from torch.optim import lr_scheduler
import time
import gc
import copy
from collections import defaultdict


import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
from PIL import Image

from segmentation.config import Configs as CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, metrics, set_seed, transforms, rle_coding
from segmentation.scr.tilling_loader import 

pd.options.mode.chained_assignment = None

C:\Users\123\AppData\Local\Temp\ipykernel_19088\30371896.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
